In [1]:
# ignore the warnings
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import warnings
warnings.filterwarnings("ignore")

In [2]:
# import the libraries needed
import pandas as pd
import networkx as nx
from model.network_creation import create_network

# Most critical road elements according to centrality metrics

In this notebook, we briefly find the most critical elements in Bangladesh road network from a topological point of view, by using the centrality metrics of the graph that we developed.

In [3]:
# prepare the data needed for the analysis
source = '../data/cleaned_roads.csv'

In [4]:
# create the network as we did in the model
network = create_network(source_csv=source)

In [5]:
# get the dataframe with the data
data = pd.read_csv(source)

Things we can analyze:
* general stuff: clustering, centrality metrics (degree, closenesss, etc.), etc. and how it relates back to the driving situation
    * betweenness centralities of bridges: the ones with the heighest are more critical?

### Computing centrality metrics for the nodes

First of all, we compute the most common centrality metrics in a network: <code>Degree</code>, <code>Closeness</code>, and <code>Betweenness</code> using the NetworkX library.

The basic description of these metrics are the following (Erath, Löchl, Axhausen, 2008):
* <code>Degree</code>: the more adjecent nodes a node has, the more important this node is
* <code>Closeness</code>: the closer a node is to all the other nodes, the more important the node
* <code>Betweenness</code>: the more shortest paths run through a node, the more imporatnt this node is

In [6]:
# create a dataframe to collect all the centrality metrics data
centrality_metrics_df_columns = ['Node id', 'Type', 'Degree', 'Closeness', 'Betweenness']
centrality_df = pd.DataFrame(columns=centrality_metrics_df_columns)
# fill with the nodes id
for node in list(network.nodes):
    centrality_df = centrality_df.append({'Node id': node, 'Type': network.nodes[node]['type'], 'Degree': None, 'Closeness': None, 'Betweenness': None}, ignore_index=True)

In [7]:
centrality_df.head()

,Node id,Type,Degree,Closeness,Betweenness
0,N1_LRPS,sourcesink,None,None,None
1,N1_link0,link,None,None,None
2,N1_LRP001a,bridge-A,None,None,None
3,N1_link1,link,None,None,None
4,N1_LRP004b,bridge-A,None,None,None


In [8]:
def add_metric_to_df(df, metric, res):
    for node in res:
        index = df.loc[df['Node id'] == node].index.tolist()[0]
        df.at[index, metric] = res[node]

In [9]:
# compute degree centrality
degree = nx.degree_centrality(network)
add_metric_to_df(centrality_df, 'Degree', degree)

In [10]:
# compute closeness centrality
closeness = nx.closeness_centrality(network)
add_metric_to_df(centrality_df, 'Closeness', closeness)

In [11]:
# compute betweenness centrality
betweenness = nx.betweenness_centrality(network)
add_metric_to_df(centrality_df, 'Betweenness', betweenness)

In [12]:
centrality_df.head()

,Node id,Type,Degree,Closeness,Betweenness
0,N1_LRPS,sourcesink,0.000517331,0.00239868,0
1,N1_link0,link,0.00103466,0.00240444,0.00103466
2,N1_LRP001a,bridge-A,0.00103466,0.00241023,0.00206825
3,N1_link1,link,0.00103466,0.00241604,0.00310077
4,N1_LRP004b,bridge-A,0.00103466,0.00242187,0.00413222


In [13]:
len(centrality_df['Degree'].unique())

3

Given the small variation in values for the <code>Degree</code> centrality because of the topology of this smaller network we are analyzing (mainly side roads only connecting to two central roads without having also the side roads connecting with one another), we can exclude the <code>Degree</code> metric from the rest of the analysis. This result is in accordance to literature (Erath, Löchl, Axhausen, 2008).

In [14]:
centrality_df = centrality_df.drop(columns=['Degree'])

### Analyzing the centrality metrics data

We are going to analyze the most critical elements of this road network, _bridges_ (because of their different conditions) and _intersections_ (because they could become bottlennecks of the general traffic flow).

#### Most critical bridges

We are going to analyze the bridges which already present a precarious condition, 'C' or 'D'.

In [15]:
# get bridges
bridges = centrality_df.loc[centrality_df['Type'].str.startswith('bridge')]

In [16]:
# add condition column
for index, row in bridges.iterrows():
    bridges.at[index, 'Condition'] = bridges.at[index, 'Type'][7] # get the last letter stored in the cell
# remove the type column
bridges = bridges.drop(columns=['Type'])
#TODO: the at operation is raising warnings! And I suppress all warnings, so watch out

In [17]:
bridges.head()

,Node id,Closeness,Betweenness,Condition
2,N1_LRP001a,0.00241023,0.00206825,A
4,N1_LRP004b,0.00242187,0.00413222,A
8,N1_LRP008b,0.00242892,0.00300116,A
10,N1_LRP010b,0.00243127,0.00341245,A
12,N1_LRP010c,0.00243362,0.00382375,A


In [18]:
# keep only the worst bridges
bridges_worst = bridges.loc[(bridges['Condition'] == 'C') | (bridges['Condition'] == 'D')]

The get the most critical bridges, we simply sort these bridges according to their centrality metrics (higer values for closeness and betweenness): these are the bridges that are more highly to break down but at the same time hold an important position in the network.

In [19]:
most_critical_betw = bridges_worst.sort_values(by=['Betweenness'], ascending=[False]).head(5)['Node id'].tolist()
print(most_critical_betw)

['N1_LRP094a', 'N1_LRP191c', 'N1_LRP208a', 'N1_LRP264c', 'N2_LRP031f']


In [20]:
most_critical_clos = bridges_worst.sort_values(by=['Closeness'], ascending=[False]).head(5)['Node id'].tolist()
print(most_critical_clos)

['N1_LRP094a', 'N1_LRP191c', 'N1_LRP208a', 'N2_LRP031f', 'N1_LRP264c']


In [21]:
most_critical = list(set(most_critical_betw) & set(most_critical_clos))
print(most_critical)

['N1_LRP094a', 'N2_LRP031f', 'N1_LRP208a', 'N1_LRP191c', 'N1_LRP264c']


Because of the topology of the network, we see that the most critical bridges have the heighest values for both <code>Closeness</code> and <code>Betweenness</code> metrics: these are the bridges that the Government of Bangladesh should invest on to the decrease the chances of major disturbances to the road network.

In [22]:
data.loc[data['id'].isin(most_critical)]

,road,id,model_type,condition,name,lat,lon,length
94,N1,N1_LRP094a,bridge,C,Mostapur Bridge,23.411932,91.193903,27.35
220,N1,N1_LRP191c,bridge,C,ISAMATI BOX CULVERT,22.713873,91.608235,19.20
264,N1,N1_LRP208a,bridge,C,BAROCONDO BRIDGE,22.587035,91.677044,22.25
360,N1,N1_LRP264c,bridge,C,AJIMPUR CUL,22.259052,92.008111,9.80
1239,N2,N2_LRP031f,bridge,C,SHIKANDI BOX CULVERT,23.895239,90.674440,2.50


#### Get the most critical intersections

Also intersections can play an important role in traffic congestion. To find the most critical ones, we are going to apply the same analysis as we did for the bridges.

In [23]:
intersections = centrality_df.loc[centrality_df['Type'] == ('intersection')]

In [29]:
most_critical_betw = intersections.sort_values(by=['Betweenness'], ascending=[False]).head(5)['Node id'].tolist()
print(most_critical_betw)

['N1_LRP084a-N102_LRPS', 'N1_LRP148a-N104_LRP001a', 'N2_LRP012b-N105_LRP012a', 'N1_LRP012c-N105_LRPS', 'N2_LRP117b-N204_LRPS']


In [30]:
most_critical_clos = intersections.sort_values(by=['Closeness'], ascending=[False]).head(5)['Node id'].tolist()
print(most_critical_clos)

['N1_LRP148a-N104_LRP001a', 'N1_LRP084a-N102_LRPS', 'N1_LRP012c-N105_LRPS', 'N2_LRP012b-N105_LRP012a', 'N1_LRP009a-N2_LRPS']


In [33]:
most_critical = list(set(most_critical_betw) & set(most_critical_clos))
print(most_critical)

['N2_LRP012b-N105_LRP012a', 'N1_LRP148a-N104_LRP001a', 'N1_LRP084a-N102_LRPS', 'N1_LRP012c-N105_LRPS']


4 of the most critical bridges according to the <code>Betweenness</code> criteria are also important in terms of <code>Closeness</code>. But since the <code>Betweenness</code> metrics relies on the shortest path and intersections are key elements to switch from one road to another if the source and destination are not on the same road, the Government of Bangaldesh should highly value mainatnace of those interesections that have the highest value for the <code>Betweenness</code> metric.

In [34]:
most_critical = most_critical_betw

In [35]:
data.loc[data['id'].isin(most_critical)][['id', 'lat', 'lon']].drop_duplicates()

,id,lat,lon
18,N1_LRP012c-N105_LRPS,23.690416,90.546597
86,N1_LRP084a-N102_LRPS,23.478972,91.118180
156,N1_LRP148a-N104_LRP001a,23.009542,91.381402
1177,N2_LRP012b-N105_LRP012a,23.785291,90.568847
1433,N2_LRP117b-N204_LRPS,24.147889,91.346527


## References

Erath, A., Löchl, M., Axhausen, K. W. (2008). Graph-Theoretical Analysis of the Swiss Road and Railway Networks Over Time. _Networks and Spatial Economics_, 9, 379–400. https://link.springer.com/article/10.1007/s11067-008-9074-7